In [8]:
from AI import TronNode, TronTree
from Map.Map import TronMap

In [9]:
def printChildrenRecursive(node, depth):
  for child in node.children:
    print("\t"*depth, child.state, ": Alpha:", child.alpha, "Beta:", child.beta, "Heuristic:", child.heuristic(), "Level:", child.level)
    if child.children:
      printChildrenRecursive(child, depth+1)

Painful stuff ahead

In [10]:
def originalTests():
  game = TronMap()
  print(game._num_players)
  print(game._current_player)
  game.print_map()
  print([game.get_player_position(1), game.get_player_position(2)])
  # Max
  treeAlphaBeta = TronTree(game, True)
  objective = treeAlphaBeta.alphaBeta(5, maxPlayer=treeAlphaBeta.root.player)
  print("Decision: ", objective.state)
  print("Children: ", end="")
  for child in objective.children:
    print(child.state, end=", ")
  print()

  game.add_move(objective.state[0][0], objective.state[0][1])

  index = 1
  printChildrenRecursive(treeAlphaBeta.root, 0)
  # Min
  #nodeInit=NodeTicTacToe(False,value="inicio",state=initState, operators= operators)
  #treeAlphaBeta= Tree(nodeInit,operators)
  #objective=treeAlphaBeta.alfaBeta(5, maxPlayer=False)

  # path=treeAlphaBeta.printPath(objective)
  # graph=treeAlphaBeta.draw(path.copy())
  # tree_image = Image(graph.create_png(), width=10000, height=10000)
  # display(tree_image)


The next thing wasn't so bad

In [11]:
from copy import deepcopy


nodes = []

def gameBetweenAI():
  nodes = []
  game = TronMap()
  print("Number of players:", game._num_players)
  game.print_map()
  print()

  currentPlayer = True
  while not game.game_finished():
    move(game, currentPlayer)
    game.print_map()
    print()

    currentPlayer = not currentPlayer

  print(game.get_players_positions())
  return game

def move(game: TronMap, player: bool):
  treeAlphaBeta = TronTree(deepcopy(game), player)
  objective = treeAlphaBeta.alphaBeta(4, maxPlayer=treeAlphaBeta.root.player)
  if player:
    game.add_move(objective.state[0][0], objective.state[0][1])
    print("Decision Max:", objective.state[0][0], ",", objective.state[0][1])
  else:
    game.add_move(objective.state[1][0], objective.state[1][1])
    print("Decision Min:", objective.state[1][0], ",", objective.state[1][1])

  nodes.append(objective)

In [14]:
game = gameBetweenAI()

Number of players: 2
[0, 2, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

Decision Max: 1 , 1
[0, 2, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

Decision Min: 2 , 0
[0, 2, 2, 0, 0, 0]
[1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

Decision Max: 2 , 1
[0, 2, 2, 0, 0, 0]
[1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

Decision Min: 3 , 0
[0, 2, 2, 2, 0, 0]
[1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

Decision Max: 3 , 1
[0, 2, 2, 2, 0, 0]
[1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

Decision Min: 4 , 0
[0, 2, 2, 2, 2, 0]
[1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

Decision Max: 4 , 1
[0, 2, 2, 2, 2, 0]
[1, 1, 1, 1, 1,

Players seem to be killing themselves if they determine that they can't win, to avoid this behaviour we can change the heuristic from setting death as - or + inf and instead add or subtract the amount of spaces the board has depending on who dies and add the node depth to the heuristic. This will effectively (or at least should) avoid making all decisions equal when death is unavoidable.

In [13]:
for node in nodes[-1].parent.children:
  print(node.state, ": Alpha:", node.alpha, "Beta:", node.beta, "Heuristic:", node.heuristic())
  printChildrenRecursive(node, 1)

[(2, 0), (0, 0)] : Alpha: -inf Beta: 39 Heuristic: 1
	 [(2, 1), (0, 0)] : Alpha: -38 Beta: inf Heuristic: -38 Level: 2
	 [(1, 0), (0, 0)] : Alpha: 39 Beta: inf Heuristic: -2 Level: 2
		 [(1, 0), (0, 1)] : Alpha: -38 Beta: 39 Heuristic: 39 Level: 3
		 [(1, 0), (1, 0)] : Alpha: -38 Beta: 39 Heuristic: 39 Level: 3
	 [(3, 0), (0, 0)] : Alpha: 39 Beta: inf Heuristic: -38 Level: 2
[(2, 0), (0, 2)] : Alpha: -inf Beta: 37 Heuristic: 37
[(2, 0), (1, 1)] : Alpha: -inf Beta: 37 Heuristic: 37


It didn't work

Maybe think of a rule for ties between nodes, like picking the one that has the highest depth?

Actually it did. But only when the alphaBeta depth is an even number.

nvm it didn't, no clue what I should try